In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Wczytanie danych o Titanicu

In [16]:
attribute_names = []
with open("Zbiór danych Titanic.arff.txt", "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip()
        if line.lower().startswith("@attribute"):
            attribute_name = line.split()[1].strip("'")
            attribute_names.append(attribute_name)

titanic_data = pd.read_csv(
    "Zbiór danych Titanic.arff.txt",
    skiprows=17,
    names=attribute_names,
    na_values=["?"])


## Analiza Kardynalności cech

#### Podsumowanie kardynalności wszytkich cech zbioru

In [17]:
headers = titanic_data.columns.tolist()
cardinality = {}
for habit in headers: 
    tmp = len(titanic_data[habit].unique())
    cardinality[habit] = tmp
    print(f'Liczba etykiet zmiennej {habit:<10}: {tmp:>5}')

Liczba etykiet zmiennej pclass    :     3
Liczba etykiet zmiennej survived  :     2
Liczba etykiet zmiennej name      :  1307
Liczba etykiet zmiennej sex       :     2
Liczba etykiet zmiennej age       :    99
Liczba etykiet zmiennej sibsp     :     7
Liczba etykiet zmiennej parch     :     8
Liczba etykiet zmiennej ticket    :   929
Liczba etykiet zmiennej fare      :   282
Liczba etykiet zmiennej cabin     :   187
Liczba etykiet zmiennej embarked  :     4
Liczba etykiet zmiennej boat      :    28
Liczba etykiet zmiennej body      :   122
Liczba etykiet zmiennej home.dest :   370


#### Liczba wszystkich pasażerów statku

In [11]:
print('Liczba wszystkich pasażerów:', len(titanic_data))
titanic_data_filt = titanic_data.drop_duplicates(subset="name", keep='first')
print('Liczba pasażerów po usunięciu duplikatów:', len(titanic_data_filt))

duplicated_rows_by_name = titanic_data[titanic_data.duplicated(subset="name", keep=False)]
print(duplicated_rows_by_name)

Liczba wszystkich pasażerów: 1309
Liczba pasażerów po usunięciu duplikatów: 1307
     pclass  survived                  name     sex   age  sibsp  parch  \
725       3         1  Connolly, Miss. Kate  female  22.0      0      0   
726       3         0  Connolly, Miss. Kate  female  30.0      0      0   
924       3         0      Kelly, Mr. James    male  34.5      0      0   
925       3         0      Kelly, Mr. James    male  44.0      0      0   

     ticket    fare cabin embarked boat  body home.dest  
725  370373  7.7500   NaN        Q   13   NaN   Ireland  
726  330972  7.6292   NaN        Q  NaN   NaN   Ireland  
924  330911  7.8292   NaN        Q  NaN  70.0       NaN  
925  363592  8.0500   NaN        S  NaN   NaN       NaN  


### Kategoryzacja kardynalności cech na dużą i małą

In [12]:
majorCardinality = [key for key, value in cardinality.items() if value > 50]
minorCardinality = [key for key, value in cardinality.items() if value <= 50]
print("Cechy o DUZEJ kardynalnosci:", end=" ")
print(", ".join(map(str, majorCardinality)))
print("Cechy o MALEJ kardynalnosci:", end=" ")
print(", ".join(map(str, minorCardinality)))

Cechy o DUZEJ kardynalnosci: name, age, ticket, fare, cabin, body, home.dest
Cechy o MALEJ kardynalnosci: pclass, survived, sex, sibsp, parch, embarked, boat


In [13]:
unique_cabins = np.unique(titanic_data['cabin'].dropna())
print(f'Liczba unikalnych kabin: {len(unique_cabins)}')
type(unique_cabins)

Liczba unikalnych kabin: 186


numpy.ndarray

### Redukcja chechy 'cabin'

In [18]:
titanic_data['CabinReduced'] = titanic_data['cabin'].astype(str).str[0]
titanic_data['CabinReduced'] = titanic_data['CabinReduced'].replace('n', np.nan)
print(titanic_data[['cabin', 'CabinReduced']].head(20))

titanic_data.to_csv("..\\cw5\\titanic_data.csv", index=False)

      cabin CabinReduced
0        B5            B
1   C22 C26            C
2   C22 C26            C
3   C22 C26            C
4   C22 C26            C
5       E12            E
6        D7            D
7       A36            A
8      C101            C
9       NaN          NaN
10  C62 C64            C
11  C62 C64            C
12      B35            B
13      NaN          NaN
14      A23            A
15      NaN          NaN
16  B58 B60            B
17  B58 B60            B
18      D15            D
19       C6            C


In [15]:
headers = titanic_data.columns.tolist()
cardinality = {}
for habit in headers: 
    tmp = len(titanic_data[habit].unique())
    cardinality[habit] = tmp
    print(f'Liczba etykiet zmiennej {habit:<15}: {tmp:>5}')

reducePercent = 100 * ((len(titanic_data['cabin'].unique()) - len(titanic_data['CabinReduced'].unique()))) / len(titanic_data['cabin'].unique())
print()
print(f'Procent zmniejszenia kardynalności kabin: {reducePercent:.2f}%')

Liczba etykiet zmiennej pclass         :     3
Liczba etykiet zmiennej survived       :     2
Liczba etykiet zmiennej name           :  1307
Liczba etykiet zmiennej sex            :     2
Liczba etykiet zmiennej age            :    99
Liczba etykiet zmiennej sibsp          :     7
Liczba etykiet zmiennej parch          :     8
Liczba etykiet zmiennej ticket         :   929
Liczba etykiet zmiennej fare           :   282
Liczba etykiet zmiennej cabin          :   187
Liczba etykiet zmiennej embarked       :     4
Liczba etykiet zmiennej boat           :    28
Liczba etykiet zmiennej body           :   122
Liczba etykiet zmiennej home.dest      :   370
Liczba etykiet zmiennej CabinReduced   :     9

Procent zmniejszenia kardynalności kabin: 95.19%


### Uzasadnienie redukcji kolumny 'cabin'

Redukcja zmiennej 'Cabin' polega na jej uproszczeniu z pełnego numeru (np. 'E12') do pierwszej litery (czyli w tym wypadku: 'E'). Prowadzi to do znacznego zmniejszenia liczby unikalnych wartości, co w dalszym ciągu może znacznie poprawić kategoryzację (imputację) brakujących wartości tej cechy.

Uproszczenie to nie wpływa w żaden znaczący sposób na utratę istotnych wniosków (danych) ze zbioru, ponieważ po pierwszej literze danej kabiny można ją skategoryzować do odpowiedniej klasy (piętra), do której ta kabina należała.

Podsumowując, zmiana ta nie wpływa w żaden negatywny sposób na nasz zbiór danych, a wręcz upraszcza go, dzięki czemu zbiór staje się bardziej czytelny i łatwiejszy do interpretacji, przy czym nie pozbywamy się żadnych ważnych informacji.